# OTA Camera (Daylight Settings)

This notebook is about exploring the ASICamera2 API for use with the ASI482MC as a camera through the Optical Tube Assembly (OTA) for remote control and exploration of the night sky.

## Includes 

In [10]:
import numpy as np
import scipy.misc
import asi
import cv2
from typing import Optional

## Identify Camera by Number

In [11]:
cameras = asi.getConnectedCameras()
print("Connected camera's:")

for camera_number in cameras:
    camera_name = (cameras[camera_number]['Name']).decode()
    print(f'  Camera Number: {camera_number} ({camera_name})')


Connected camera's:
  Camera Number: 0 (ZWO ASI120MC-S(Camera))
  Camera Number: 1 (ZWO ASI482MC)


### Playing with exposure and gain settings based on use (Finder Camera).
Natural daylight: exposure around 300,000 us. gain is 0

Night observing (decent image feedback): exposure around 2,000,000 us. gain around 80

Night observing (long exposure): exposure around 20,000,000 us. gain around 50

In [5]:


class VideoCamera(object):
    _video = None
    _exposure = 3400 #2000000
    _offset_cross = (-12,-8)
    _gain = 0 #82
    last_image: Optional[bytes] = None

    def __init__(self):
        self._video = asi.Camera(1)
        
        self._video.setCaptureFrameFormat(1920, 1080, 1, "RGB24")
        self._video.setControlValueManual("ASI_GAIN", self._gain)
        self._video.setControlValueManual("ASI_WB_R", 55)
        self._video.setControlValueManual("ASI_WB_B", 80)

    def __del__(self):
        del self._video

    def get_frame(self) -> bytes:
        self.last_image, bin, success = self._video.grab(self._exposure)
        b,g,r = cv2.split(self.last_image)
        corrected_image = cv2.merge ( (r, g, b) )

        png = corrected_image[80-self._offset_cross[0]:880-self._offset_cross[0],240-self._offset_cross[1]:1040-self._offset_cross[1]]
        
        ret, jpeg = cv2.imencode('.jpg', png)
        return jpeg.tobytes()



In [6]:
import uvicorn
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import threading

app = FastAPI()

camera: VideoCamera  = None

def run():
    uvicorn.run(app, host="0.0.0.0", port=8001)
      
def start_api():
    _api_thread = threading.Thread(target=run)
    _api_thread.start()
    
def create_stream():
    global camera
    camera = VideoCamera()    
    while True:
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + bytearray(camera.get_frame()) + b'\r\n\r\n')
      
@app.get("/")
def get_video_stream():
    return StreamingResponse(create_stream(), media_type='multipart/x-mixed-replace; boundary=frame')

start_api()

In [7]:
%%HTML 
<img src="http://192.168.0.30:8001/" width=640 />

In [56]:
print(camera._video.setControlValueManual("ASI_WB_R",55))
print(camera._video.setControlValueManual("ASI_WB_B",88))


{'value': 55, 'auto': False}
{'value': 88, 'auto': False}


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 404, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 269, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-packa